In [2]:
import pandas as pd
import csv
import math
import numpy as np # library to handle data in a vectorized manner
import folium # map rendering library

import matplotlib.pyplot as plt # plotting library
# backend for rendering plots within the browser
%matplotlib inline 

from sklearn.cluster import KMeans

import matplotlib.cm as cm
import matplotlib.colors as colors


print('Libraries imported.')


Libraries imported.


In [4]:
FILENAME='scraped_venues_tiled.csv'

df_all=pd.read_csv(FILENAME)
df_all.drop_duplicates(subset='Id',inplace=True)
print('{} unique venues in total'.format(len(df_all)))

lat_list=df_all['Latitude'].tolist()
lng_list=df_all['Longtitude'].tolist()

lng_list_equidistant=[]

for i in range(0, len(lat_list)):
    lng_list_equidistant.append(lng_list[i]*math.cos(math.radians(lat_list[i]))) 
    
df_all['Longtitude_equidistant'] = lng_list_equidistant
df_all



99073 unique venues in total


,Name,Id,Latitude,Longtitude,Categories,City,Longtitude_equidistant
0,The Wisley,4bf9195c5317a5937abf007f,51.318840,-0.482647,Golf Course,"London, England-0-0",-0.301648
1,Ockham Bites,4c172c535e88c92829422566,51.316893,-0.453541,Café,"London, England-0-0",-0.283469
2,Pyrford Golf Club,4c557759a724e21ee3fc1ef8,51.315947,-0.494943,Golf Course,"London, England-0-0",-0.309352
3,Terrace Restaurant,57385044498ed9b3efb33dae,51.315352,-0.473335,Restaurant,"London, England-0-0",-0.295850
4,Food Court,5d481c23816c150008c6a819,51.315492,-0.473591,Food Court,"London, England-0-0",-0.296009
5,Ockham And Wisley Commons,4cca9a8db7d9a1438faffd36,51.315512,-0.454796,Park,"London, England-0-0",-0.284262
6,Glasshouse Café,4c1f4ce0eac020a1eed74ac2,51.314778,-0.478156,Café,"London, England-0-0",-0.298867
7,Wisley Shop,51b0b4ab498eb2b09f8b5974,51.314674,-0.471898,Gift Shop,"London, England-0-0",-0.294956
8,The Glasshouse,4b6ebdb8f964a52094c82ce3,51.314204,-0.478711,Garden,"London, England-0-0",-0.299218
9,RHS Garden Wisley,4b938c5ff964a520d94734e3,51.313881,-0.473204,Botanical Garden,"London, England-0-0",-0.295778


In [5]:
n_clusters = 991

k_means_all = KMeans(n_clusters = n_clusters, verbose=2 )
k_means_all.fit(df_all[['Latitude','Longtitude_equidistant']])

df_all['Neighborhood_label'] = k_means_all.labels_

df_all

Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 10.091729612859323
start iteration
done sorting
end inner loop
Iteration 1, inertia 9.561858944416946
start iteration
done sorting
end inner loop
Iteration 2, inertia 9.350325484906971
start iteration
done sorting
end inner loop
Iteration 3, inertia 9.241719123713064
start iteration
done sorting
end inner loop
Iteration 4, inertia 9.163809965048864
start iteration
done sorting
end inner loop
Iteration 5, inertia 9.111543598906147
start iteration
done sorting
end inner loop
Iteration 6, inertia 9.0740974830468
start iteration
done sorting
end inner loop
Iteration 7, inertia 9.042401294225018
start iteration
done sorting
end inner loop
Iteration 8, inertia 9.017826809107627
start iteration
done sorting
end inner loop
Iteration 9, inertia 8.996405325636971
start iteration
done sorting
end inner loop
Iteration 10, inertia 8.982589421281135
start iteration
done sorting
end inner loop
Iteration 11, inert

,Name,Id,Latitude,Longtitude,Categories,City,Longtitude_equidistant,Neighborhood_label
0,The Wisley,4bf9195c5317a5937abf007f,51.318840,-0.482647,Golf Course,"London, England-0-0",-0.301648,599
1,Ockham Bites,4c172c535e88c92829422566,51.316893,-0.453541,Café,"London, England-0-0",-0.283469,763
2,Pyrford Golf Club,4c557759a724e21ee3fc1ef8,51.315947,-0.494943,Golf Course,"London, England-0-0",-0.309352,599
3,Terrace Restaurant,57385044498ed9b3efb33dae,51.315352,-0.473335,Restaurant,"London, England-0-0",-0.295850,599
4,Food Court,5d481c23816c150008c6a819,51.315492,-0.473591,Food Court,"London, England-0-0",-0.296009,599
5,Ockham And Wisley Commons,4cca9a8db7d9a1438faffd36,51.315512,-0.454796,Park,"London, England-0-0",-0.284262,240
6,Glasshouse Café,4c1f4ce0eac020a1eed74ac2,51.314778,-0.478156,Café,"London, England-0-0",-0.298867,599
7,Wisley Shop,51b0b4ab498eb2b09f8b5974,51.314674,-0.471898,Gift Shop,"London, England-0-0",-0.294956,599
8,The Glasshouse,4b6ebdb8f964a52094c82ce3,51.314204,-0.478711,Garden,"London, England-0-0",-0.299218,599
9,RHS Garden Wisley,4b938c5ff964a520d94734e3,51.313881,-0.473204,Botanical Garden,"London, England-0-0",-0.295778,599


In [7]:
london_rows = df_all.City.str.contains('London')
manchester_rows = df_all.City.str.contains('Manchester')
birmingham_rows = df_all.City.str.contains('Birmingham')
berlin_rows = df_all.City.str.contains('Berlin')
rome_rows = df_all.City.str.contains('Rome')


df_london = df_all[london_rows].reset_index(drop=True)
df_manchester = df_all[manchester_rows].reset_index(drop=True)
df_birmingham = df_all[birmingham_rows].reset_index(drop=True)
df_berlin = df_all[berlin_rows].reset_index(drop=True)
df_rome = df_all[rome_rows].reset_index(drop=True)

print('Scraped venue counts:')
print('London: {}'.format(len(df_london)))
print('Manchester: {}'.format(len(df_manchester)))
print('Birmingham: {}'.format(len(df_birmingham)))
print('Berlin: {}'.format(len(df_berlin)))
print('Rome: {}'.format(len(df_rome)))





Scraped venue counts:
London: 40635
Manchester: 12296
Birmingham: 11570
Berlin: 19713
Rome: 14859


In [ ]:
mean_latitude_list_berlin=df_berlin.groupby('Neighborhood_label').mean()['Latitude'].tolist()
mean_longtitude_list_berlin=df_berlin.groupby('Neighborhood_label').mean()['Longtitude'].tolist()

import statistics

mean_latitude_berlin=statistics.mean(mean_latitude_list_berlin)
mean_longtitude_berlin=statistics.mean(mean_longtitude_list_berlin)



In [ ]:
map_clusters = folium.Map(location=[mean_latitude_berlin, mean_longtitude_berlin], zoom_start=11)

for lat,lng in zip(mean_latitude_list_berlin, mean_longtitude_list_berlin):
    
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_clusters)

for lat,lng in zip(df_berlin['Latitude'].head(2000), df_berlin['Longtitude'].head(2000)):
    
    folium.CircleMarker(
        [lat, lng],
        radius=2,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_clusters)    
    

map_clusters

In [ ]:
mean_latitude_list_rome=df_rome.groupby('Neighborhood_label').mean()['Latitude'].tolist()
mean_longtitude_list_rome=df_rome.groupby('Neighborhood_label').mean()['Longtitude'].tolist()

import statistics

mean_latitude_rome=statistics.mean(mean_latitude_list_rome)
mean_longtitude_rome=statistics.mean(mean_longtitude_list_rome)

map_clusters_rome = folium.Map(location=[mean_latitude_rome, mean_longtitude_rome], zoom_start=11)

for lat,lng in zip(mean_latitude_list_rome, mean_longtitude_list_rome):
    
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_clusters_rome)

for lat,lng in zip(df_rome['Latitude'].head(2000), df_rome['Longtitude'].head(2000)):
    
    folium.CircleMarker(
        [lat, lng],
        radius=2,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_clusters_rome)    
    

map_clusters_rome

In [ ]:
mean_latitude_list_london=df_london.groupby('Neighborhood_label').mean()['Latitude'].tolist()
mean_longtitude_list_london=df_london.groupby('Neighborhood_label').mean()['Longtitude'].tolist()

import statistics

mean_latitude_london=statistics.mean(mean_latitude_list_london)
mean_longtitude_london=statistics.mean(mean_longtitude_list_london)

map_clusters_london = folium.Map(location=[mean_latitude_london, mean_longtitude_london], zoom_start=11)

for lat,lng in zip(mean_latitude_list_london, mean_longtitude_list_london):
    
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_clusters_london)

for lat,lng in zip(df_london['Latitude'].head(2000), df_london['Longtitude'].head(2000)):
    
    folium.CircleMarker(
        [lat, lng],
        radius=2,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_clusters_london)    
    

map_clusters_london


In [9]:
df_london.to_csv('clustered_london.csv',index=False)
df_manchester.to_csv('clustered_manchester.csv',index=False)
df_birmingham.to_csv('clustered_birmigham.csv',index=False)
df_berlin.to_csv('clustered_berlin.csv',index=False)
df_rome.to_csv('clustered_rome.csv',index=False)
df_all.to_csv('clustered_all.csv',index=False)

df_uk = pd.concat([df_london, df_manchester, df_birmingham])
df_capitals = pd.concat([df_london, df_berlin, df_rome])
df_b2uk = pd.concat([df_berlin, df_birmingham, df_manchester])

df_uk.to_csv('clustered_uk.csv',index=False)
df_capitals.to_csv('clustered_capitals.csv',index=False)
df_b2uk.to_csv('clustered_b2uk.csv',index=False)

